In [ ]:
import pandas as pd
df = pd.read_csv('queryset_CNN.csv')
print(df.shape)
print(df.dtypes)

(32, 3)
doc_id         int64
author_id      int64
authorList    object
dtype: object


In [ ]:

for index, row in df.iterrows():
    doc_id = row.doc_id

    author_id = row.author_id

    import ast
    authorList = ast.literal_eval(row.authorList)
    
    candidate = len(authorList)

    test = "lstm" # change before run

    level = "word"

    iterations = 30

    dropout = 0.5

    samples = 3200

    dimensions = 200

    loc = authorList.index(author_id)

    printstate = (("doc_id = %s, candidate = %s, ") % (str(doc_id), str(candidate)))
    printstate += (("dimensions = %s, samples = %s, ") % (str(dimensions), str(samples)))
    printstate += (("\niterations = %s, dropout = %s, test = %s") % (str(iterations), str(dropout), str(test)))

    print("Current test: %s" % (str(printstate)))
    
    import UpdateDB as db
    case = db.checkCNN(doc_id = doc_id, candidate = candidate, dimensions = dimensions,
                       samples = samples, iterations = iterations, dropout = dropout,
                       test = test)

    if case == False:

        print("Running: %12s" % (str(printstate)))

        import StyloNeuralLSTM as Stylo
        (labels_index, history, train_acc, val_acc, samples) = Stylo.getResults(
            doc_id = doc_id, authorList = authorList[:], 
            level = level, glove = '../../glove/', dimensions = dimensions, 
            samples = samples, nb_epoch = iterations, dropout = dropout, batch_size = 10 )

        (predYList, predY, testY) = Stylo.getTestResults(
            doc_id = doc_id, authorList = authorList[:], labels_index = labels_index,
            level = level, glove = '../../glove/', dimensions = dimensions, 
            samples = samples, nb_epoch = iterations, dropout = dropout, batch_size = 10 )

        loc = testY
        
        test_acc = predY[loc]

        test_bin = 0

        if(predY.tolist().index(max(predY)) == testY):
            test_bin = 1
        
        import UpdateDB as db
        case = db.updateresultCNN(doc_id = doc_id, candidate = candidate, dimensions = dimensions,
                                  samples = samples, iterations = iterations, dropout = dropout,
                                  train_acc = train_acc, val_acc = val_acc,
                                  test_acc = test_acc, test_bin = test_bin,
                                  test = test)
                                     
        del Stylo

        from keras import backend as K
        K.clear_session()

        import time
        time.sleep(10)
        
        from IPython.display import clear_output

        clear_output()

    else:
        print("Skipped: %12s" % (str(printstate)))


# import pandas as pd
# df = pd.DataFrame(output)
# df.to_csv("styloout.csv", index = False, encoding='utf-8')

import time
time.sleep(10)

Current test: doc_id = 85, candidate = 3, dimensions = 200, samples = 3200, 
iterations = 30, dropout = 0.5, test = batch10
Execution completed
Running: doc_id = 85, candidate = 3, dimensions = 200, samples = 3200, 
iterations = 30, dropout = 0.5, test = batch10


Using TensorFlow backend.


Level = Word
File used: glove.6B.200d.txt
Found 400000 word vectors.
Execution completed
Read completed
Number of rows: 124
author_id       int64
doc_content    object
dtype: object
Data Frame created: Shape: (11577, 2)
Author:    44  Size:  4745
Author:    64  Size:  5106
Author:    82  Size:  1726
Min: 1726
Max: 5106
Authors [44, 64, 82].
Found 5178 texts.
Found 5178 labels.
Found 46419 unique tokens.
Shape of data tensor: (5178, 1000)
Shape of label tensor: (5178, 3)
Done compiling.
Train on 4142 samples, validate on 1036 samples
Epoch 1/30
4142/4142 [==============================] - 46s - loss: 0.7853 - acc: 0.6352 - val_loss: 0.4702 - val_acc: 0.8243
Epoch 2/30
4142/4142 [==============================] - 43s - loss: 0.3360 - acc: 0.8747 - val_loss: 0.2415 - val_acc: 0.9218
Epoch 3/30
4142/4142 [==============================] - 42s - loss: 0.1917 - acc: 0.9310 - val_loss: 0.4019 - val_acc: 0.8465
Epoch 4/30
4142/4142 [==============================] - 42s - loss: 0.1327 - acc: 0

In [ ]:
%tb